https://docs.langchain.com/oss/python/langchain/rag#huggingface-2

In [1]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

In [9]:
"""from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")"""

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [11]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

file_paht = "data/"

loader = PyPDFDirectoryLoader(file_paht)

docs = loader.load()

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents")

Split blog post into 3428 sub-documents


In [13]:
document_ids = vector_store.add_documents(documents=all_splits)
vector_store.dump("data/nedical_book_vector_db.json")

In [14]:
vector_store = InMemoryVectorStore.load("data/nedical_book_vector_db.json", embedding=embeddings)

In [15]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import create_agent


def generate_system_message(user_query)->str:
    retrieved_docs = vector_store.similarity_search(user_query)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    system_message = (
        "Yor are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )
    return system_message

agent = create_agent(model)


In [18]:
query = "What is IBD and its reasons?"

resulst = agent.invoke({"messages": [{"role":"system", "content":generate_system_message(query)},{"role":"user", "content":query}]})
print(resulst["messages"][-1].content)


IBD stands for **Inflammatory Bowel Disease**. It is a condition in which the lining of the intestine becomes inflamed.

The provided text does not specify the exact reasons or causes for IBD. It only mentions that IBD can produce symptoms similar to amebiasis and that it's crucial to distinguish between the two because treatments for IBD can worsen amebiasis.
